In [1]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss

from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import numpy as np # linear algebra
import pandas as pd #

Using TensorFlow backend.


In [2]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid','qtopic','qtitlec','qinfoc']]

In [3]:
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])
member_info=member_info[['uid','l_topic','topic_n']]

In [4]:
def split_topic(x):
    x=x.split(',')
    return ','.join([i.split(':')[0] for i in x])
member_info['u_topic']=member_info['topic_n'].apply(lambda x:split_topic(x))

In [ ]:
def split_weight(x):
    if x!='-1':
        x=x.split(',')
        return [float(i.split(':')[1]) for i in x]
    else:return [0]*10
member_info['w_topic']=member_info['topic_n'].apply(lambda x:split_weight(x))


In [ ]:
member_info.loc[member_info['l_topic']=='-1','l_topic']=member_info[member_info['l_topic']=='-1']['u_topic']

In [ ]:
last_answer=pd.read_hdf('last_answerlist_6.h5', key='data')

In [ ]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
t_shape=len(train)
import gc
gc.collect()

In [ ]:
member_info.drop(['topic_n'],axis=1,inplace=True)

In [ ]:
ques_info_temp=ques_info[['qid','qtopic']]

In [ ]:
last_answer=last_answer.merge(ques_info_temp,on='qid',how='left')
ques_info_temp.columns=['qid_1','qtopic_1']
last_answer=last_answer.merge(ques_info_temp,on='qid_1',how='left')
ques_info_temp.columns=['qid_2','qtopic_2']
last_answer=last_answer.merge(ques_info_temp,on='qid_2',how='left')
ques_info_temp.columns=['qid_3','qtopic_3']
last_answer=last_answer.merge(ques_info_temp,on='qid_3',how='left')
ques_info_temp.columns=['qid_4','qtopic_4']
last_answer=last_answer.merge(ques_info_temp,on='qid_4',how='left')
ques_info_temp.columns=['qid_5','qtopic_5']
last_answer=last_answer.merge(ques_info_temp,on='qid_5',how='left')
ques_info_temp.columns=['qid_6','qtopic_6']
last_answer=last_answer.merge(ques_info_temp,on='qid_6',how='left')

In [ ]:
import pickle
with open('esim_topic_tokenizer.pickle', 'rb') as f:
      tokenizer = pickle.load(f)

In [ ]:
last_answer.drop(['qid_1','qid_2','qid_3','qid_4','qid_5','qid_6'],axis=1,inplace=True)

In [ ]:
last_answer=last_answer.merge(member_info,on='uid',how='left')

In [ ]:
last_answer=last_answer.fillna('-1')

In [ ]:
last_answer.columns

In [ ]:
qtopic  =tokenizer.texts_to_sequences(last_answer['qtopic'])
qtopic_1=tokenizer.texts_to_sequences(last_answer['qtopic_1'])
qtopic_2=tokenizer.texts_to_sequences(last_answer['qtopic_2'])
qtopic_3=tokenizer.texts_to_sequences(last_answer['qtopic_3'])
qtopic_4=tokenizer.texts_to_sequences(last_answer['qtopic_4'])
qtopic_5=tokenizer.texts_to_sequences(last_answer['qtopic_5'])
qtopic_6=tokenizer.texts_to_sequences(last_answer['qtopic_6'])

In [ ]:
l_topic=tokenizer.texts_to_sequences(last_answer['l_topic'])
u_topic=tokenizer.texts_to_sequences(last_answer['u_topic'])

In [ ]:
qtopic=pad_sequences(qtopic,maxlen=10,value=0)
qtopic_1=pad_sequences(qtopic_1,maxlen=10,value=0)
qtopic_2=pad_sequences(qtopic_2,maxlen=10,value=0)
qtopic_3=pad_sequences(qtopic_3,maxlen=10,value=0)
qtopic_4=pad_sequences(qtopic_4,maxlen=10,value=0)
qtopic_5=pad_sequences(qtopic_5,maxlen=10,value=0)
qtopic_6=pad_sequences(qtopic_6,maxlen=10,value=0)

In [ ]:
l_topic=pad_sequences(l_topic,maxlen=50,value=0)
u_topic=pad_sequences(u_topic,maxlen=10,value=0)

In [ ]:
np.save("qtopic.npy", qtopic)
np.save("qtopic_1.npy", qtopic_1)
np.save("qtopic_2.npy", qtopic_2)
np.save("qtopic_3.npy", qtopic_3)
np.save("qtopic_4.npy", qtopic_4)
np.save("qtopic_5.npy", qtopic_5)
np.save("qtopic_6.npy", qtopic_6)
np.save("l_topic.npy", l_topic)
np.save("u_topic.npy", u_topic)


In [ ]:
##############################################################################################

In [2]:
ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
ques_info=ques_info[['qid','qtitlec']]

In [3]:
last_answer=pd.read_hdf('last_answerlist_6.h5', key='data')

In [4]:
last_answer=last_answer.merge(ques_info,on='qid',how='left')
ques_info.columns=['qid_1','qtitlec_1']
last_answer=last_answer.merge(ques_info,on='qid_1',how='left')
ques_info.columns=['qid_2','qtitlec_2']
last_answer=last_answer.merge(ques_info,on='qid_2',how='left')
ques_info.columns=['qid_3','qtitlec_3']
last_answer=last_answer.merge(ques_info,on='qid_3',how='left')
ques_info.columns=['qid_4','qtitlec_4']
last_answer=last_answer.merge(ques_info,on='qid_4',how='left')
ques_info.columns=['qid_5','qtitlec_5']
last_answer=last_answer.merge(ques_info,on='qid_5',how='left')
ques_info.columns=['qid_6','qtitlec_6']
last_answer=last_answer.merge(ques_info,on='qid_6',how='left')

In [5]:
import pickle
with open('esim_word_tokenizer.pickle', 'rb') as f:
      tokenizer = pickle.load(f)

In [6]:
last_answer=last_answer.fillna('-1')

In [7]:
qtitlec  =tokenizer.texts_to_sequences(last_answer['qtitlec'])
qtitlec_1=tokenizer.texts_to_sequences(last_answer['qtitlec_1'])
qtitlec_2=tokenizer.texts_to_sequences(last_answer['qtitlec_2'])
qtitlec_3=tokenizer.texts_to_sequences(last_answer['qtitlec_3'])
qtitlec_4=tokenizer.texts_to_sequences(last_answer['qtitlec_4'])
qtitlec_5=tokenizer.texts_to_sequences(last_answer['qtitlec_5'])
qtitlec_6=tokenizer.texts_to_sequences(last_answer['qtitlec_6'])

In [8]:
qtitlec=pad_sequences(qtitlec,maxlen=10,value=0)
qtitlec_1=pad_sequences(qtitlec_1,maxlen=10,value=0)
qtitlec_2=pad_sequences(qtitlec_2,maxlen=10,value=0)
qtitlec_3=pad_sequences(qtitlec_3,maxlen=10,value=0)
qtitlec_4=pad_sequences(qtitlec_4,maxlen=10,value=0)
qtitlec_5=pad_sequences(qtitlec_5,maxlen=10,value=0)
qtitlec_6=pad_sequences(qtitlec_6,maxlen=10,value=0)

In [9]:
np.save("qtitlec.npy", qtitlec)
np.save("qtitlec_1.npy", qtitlec_1)
np.save("qtitlec_2.npy", qtitlec_2)
np.save("qtitlec_3.npy", qtitlec_3)
np.save("qtitlec_4.npy", qtitlec_4)
np.save("qtitlec_5.npy", qtitlec_5)
np.save("qtitlec_6.npy", qtitlec_6)